# Manage Experiment Relationships

``rubicon-ml`` experiments can be tagged with special identifiers to denote a parent/child relationship.
This can be used to track hierarchical or iterative experiments, among other things.

First, let's create a project.

In [1]:
from rubicon_ml import Rubicon

rubicon = Rubicon(persistence="memory")
project = rubicon.create_project(name="hierarchical experiments")
project

## Hierarchical experiments

Now we can log some experiments in a nested loop. Imagine logging an experiment for each node of a
gradient boosted tree, or something along those lines.

We can use ``parent_experiment.add_child_experiment(child_experiment)`` to automatically add tags
to both ``parent_experiment`` and ``child_experiment`` that represent their relationship.

In [2]:
root_experiment = project.log_experiment(name="root")

for n in range(3):
    node_experiment = project.log_experiment(name=f"node_{n}")
    root_experiment.add_child_experiment(node_experiment)

    for m in range(2):
        nested_node_experiment = project.log_experiment(name=f"node_{n}_{m}")
        node_experiment.add_child_experiment(nested_node_experiment)

To retrieve experiments, start at the root experiment and call ``get_child_experiments`` to return a
list of ``rubicon-ml`` objects representing each of the tagged child experiments.

In [3]:
for experiment in root_experiment.get_child_experiments():
    print("id:", experiment.id)
    print("tags:", [t for t in experiment.tags if "child" in t], "\n")

    for nested_experiment in experiment.get_child_experiments():
        print("\tid:", nested_experiment.id)
        print("\ttags:", nested_experiment.tags, "\n")

id: 12fac8d7-ce2a-4229-a545-286aa2539609
tags: ['child:c2db7703-d7a8-4e50-bd88-445ca1cb1410', 'child:fff24ae1-fa82-4526-bac9-d8d27e58f1f7'] 

	id: c2db7703-d7a8-4e50-bd88-445ca1cb1410
	tags: ['parent:12fac8d7-ce2a-4229-a545-286aa2539609'] 

	id: fff24ae1-fa82-4526-bac9-d8d27e58f1f7
	tags: ['parent:12fac8d7-ce2a-4229-a545-286aa2539609'] 

id: 74c3c5f2-b201-4460-a250-d42c29fa799f
tags: ['child:f1531c94-e626-4600-b018-cbb41d29e861', 'child:c729eb4c-7131-4339-8e91-ac6e786aafe6'] 

	id: f1531c94-e626-4600-b018-cbb41d29e861
	tags: ['parent:74c3c5f2-b201-4460-a250-d42c29fa799f'] 

	id: c729eb4c-7131-4339-8e91-ac6e786aafe6
	tags: ['parent:74c3c5f2-b201-4460-a250-d42c29fa799f'] 

id: 7ab02de9-a07b-4394-983a-e1cb17f16cb4
tags: ['child:d54e32d8-c1ac-4182-9a74-d07da0b5980a', 'child:3b47be81-7a1e-49be-ad9c-d48070fa06ba'] 

	id: d54e32d8-c1ac-4182-9a74-d07da0b5980a
	tags: ['parent:7ab02de9-a07b-4394-983a-e1cb17f16cb4'] 

	id: 3b47be81-7a1e-49be-ad9c-d48070fa06ba
	tags: ['parent:7ab02de9-a07b-4394-98

## Iterative experiments

We can leverage ``add_child_experiment`` to maintain iterative relationships too. This could be
used to log metadata about of each iteration of recursive feature elimination and preserve the
linear history of the model training.

In [4]:
current_experiment = project.log_experiment(name="experiment_0")

for n in range(3):
    next_experiment = project.log_experiment(name=f"experiment_{n+1}")
    current_experiment.add_child_experiment(next_experiment)

    current_experiment = next_experiment

last_experiment = current_experiment

Similarly to ``get_child_experiments``, we can use ``get_parent_experiment`` to return a ``rubicon-ml``
object representing the tagged parent experiment.

In [5]:
experiment = last_experiment

while experiment is not None:
    print("name:", experiment.name)
    print("\tid:", experiment.id)
    print("\ttags:", experiment.tags, "\n")

    experiment = experiment.get_parent_experiment()

name: experiment_3
	id: 1d8d3f84-44bc-423b-9d41-27193e948732
	tags: ['parent:2ace6121-91b8-435b-a786-9b8bc8615430'] 

name: experiment_2
	id: 2ace6121-91b8-435b-a786-9b8bc8615430
	tags: ['parent:60bda2d6-5f7d-4c10-a2be-a09d00ce7a9c', 'child:1d8d3f84-44bc-423b-9d41-27193e948732'] 

name: experiment_1
	id: 60bda2d6-5f7d-4c10-a2be-a09d00ce7a9c
	tags: ['child:2ace6121-91b8-435b-a786-9b8bc8615430', 'parent:785a1d46-eddd-44a5-8be8-4d07662f2af2'] 

name: experiment_0
	id: 785a1d46-eddd-44a5-8be8-4d07662f2af2
	tags: ['child:60bda2d6-5f7d-4c10-a2be-a09d00ce7a9c'] 

